In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, chi2_contingency, f_oneway
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_csv('../data/processed/clean_dataset.csv')
data

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000093,31520,389,2015-04-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000094,31520,389,2015-06-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000095,31520,389,2015-08-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Third Party,Third Party,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,347.235175,0.0
1000096,31519,389,2014-07-01,False,ZW,Individual,Mr,English,ABSA Bank,Savings account,...,No excess,Passenger Liability,Passenger Liability,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,2.315000,0.0


In [3]:
# Filter out rows with TotalPremium == 0
data = data[data['TotalPremium'] > 0]
print(f"Remaining rows after filtering: {data.shape[0]}")

Remaining rows after filtering: 618176


In [4]:
# Calculate KPIs
data['ClaimRatio'] = data['TotalClaims'] / data['TotalPremium']
data['Margin'] = data['TotalPremium'] - data['TotalClaims']

/tmp/ipykernel_17619/2260156487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ClaimRatio'] = data['TotalClaims'] / data['TotalPremium']
/tmp/ipykernel_17619/2260156487.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Margin'] = data['TotalPremium'] - data['TotalClaims']


In [9]:
data[['TotalClaims', 'TotalPremium', 'ClaimRatio']].describe()

,TotalClaims,TotalPremium,ClaimRatio
count,618176.000000,618176.000000,618176.000000
mean,100.413265,100.203033,0.350048
std,2989.980798,286.236094,9.288639
min,-12002.412281,0.000011,-18.700122
25%,0.000000,2.631316,0.000000
50%,0.000000,7.281228,0.000000
75%,0.000000,78.947368,0.000000
max,393092.105263,65282.603421,2553.600000


In [10]:
# Filter out rows with negative claims
data = data[data['TotalClaims'] >= 0]

In [12]:
# Filter out rows with TotalPremium below a threshold
data = data[data['TotalPremium'] > 1]

In [13]:
data[['TotalClaims', 'TotalPremium', 'ClaimRatio']].describe()

,TotalClaims,TotalPremium,ClaimRatio
count,596328.000000,596328.000000,596328.000000
mean,104.121832,103.847979,0.362921
std,3044.148472,290.784655,9.456977
min,0.000000,1.000047,0.000000
25%,0.000000,3.115263,0.000000
50%,0.000000,9.634211,0.000000
75%,0.000000,78.947368,0.000000
max,393092.105263,65282.603421,2553.600000


In [15]:
# Group data by Province
province_data = data[['Province', 'ClaimRatio']].dropna()

# Perform ANOVA
province_groups = [province_data[province_data['Province'] == province]['ClaimRatio'] for province in province_data['Province'].unique()]

# One-way ANOVA
f_stat, p_value = f_oneway(*province_groups)

print("Risk Differences Across Provinces:")
print(f"F-statistic: {f_stat}, p-value: {p_value}")

if p_value < 0.05:
    print("Reject H₀₁: There are significant risk differences across provinces.")
else:
    print("Fail to Reject H₀₁: No significant risk differences across provinces.")


Risk Differences Across Provinces:
F-statistic: 4.903759286544888, p-value: 4.457128070776272e-06
Reject H₀₁: There are significant risk differences across provinces.


In [16]:
# Group data by PostalCode
zip_data = data[['PostalCode', 'ClaimRatio']].dropna()

# Perform t-test for two selected zip codes as an example
group_a = zip_data[zip_data['PostalCode'] == 1459]['ClaimRatio']
group_b = zip_data[zip_data['PostalCode'] == 2000]['ClaimRatio']

t_stat, p_value = ttest_ind(group_a, group_b, equal_var=False)

print("\nRisk Differences Between Zip Codes:")
print(f"T-statistic: {t_stat}, p-value: {p_value}")

if p_value < 0.05:
    print("Reject H₀₂: There are significant risk differences between zip codes.")
else:
    print("Fail to Reject H₀₂: No significant risk differences between zip codes.")


Risk Differences Between Zip Codes:
T-statistic: -11.841732963674227, p-value: 2.513925500679008e-32
Reject H₀₂: There are significant risk differences between zip codes.
